# Sprawozdanie z projektu
### Autor
Marcin Bździuch, 99367

## Cel projektu
Celem projektu było stworzenie dwóch klasyfikatorów dla zbioru danych z Protein Data Bank. Pierwszy klasyfikator operował na oryinalnych klasach (nazwach ligandu), drugi na klasach grupowych (zgrupowane nazwy ligandu).

## Narzędzia
Do realizacji projektu wykorzystano:

* Język Python 2.7.10
* Bibliotekę pandas
* Bibliotekę scikit-learn
* IPython Notebook

## Użycie aplikacji
W efekcie końcowym powstał skrypt języka Python `main.py`, który dokonuje wczytania i oczyszczenia danych z pliku CSV (`preprocess`), wyszukania najlepszego klasyfikatora dla klas oryginalnych (`search`) i klas grupowych (`searchg`) oraz dokonania klasyfikacji zbioru testowego za pomocą pierwszego klasyfikatora (`classify`) i drugiego (`classifyg`).

Użycie:
`python main.py OPCJE...`

| parametr | znaczenie |
| ---------|-----------|
| `data preprocess` | Wczytanie danych z pliku `all_summary.txt` i oczyszczenie danych. Wynikiem jest zserializowany w pliku `cache/df` DataFrame  zawierający oczyszczone dane. |
| `test preprocess` | To samo co `data preprocess`, tylko dane zostaną wczytane z pliku `test_summary.txt`. Opcja służy do testowania skryptu na mniejszym (testowym) zbiorze danych. |
| `search` | Wczytanie DataFrame `cache/df` z danymi i wyszukanie najlepszego klasyfikatora operującego na klasach oryginalnych. Wynikiem jest klasyfikator zserializowany w pliku `cache/es`. |
| `searchg` | To samo co `search` tylko wynikowy klasyfikator operuje na klasach grupowych wczytanych z pliku `grouped_res_name.txt`. Wynikiem jest klasyfikator zserializowany w pliku `cache/esg`. |
| `classify` | Wczytuje klasyfikator `cache/es`, DataFrame `cache/df` i dane testowe z pliku `test_data.txt`, które następnie są poddawane klasyfikacji. Wynikiem jest wektor klas zapisany w pliku `results/classify.txt`. |
| `classifyg` | To samo co `classify` tylko działający na klasyfikatorze dla klas grupowych `cache/esg`. Wynikiem jest wektor klas grupowych zapisany w pliku `results/classifyg.txt`. |

Opcje można dowolnie łączyć w łańcuchy, np.:

`python main.py data preprocess search classify`

spowoduje wykonanie wszystkich czynności: wczytanie i oczyszczenie danych, wyszukanie klasyfikatora i klasyfikację zbioru testowego.

`python main.py data preprocess search searchg classify classifyg`

spowoduje wykonanie wszystkich czynności dla obu klasyfikatorów.

**UWAGA** - przed użyciem aplikacji należy wypakować wszystkie pliki z danymi z archiwów `all_summary.7z` i `test_data.7z`.

## Opis kroków

### Wczytanie i oczyszczenie danych
Pierwszym krokiem było wczytanie i oczyszczenie danych wejściowych.

In [ ]:
def preprocess_data():
	file = 'all_summary.txt'
	df = read_data(file, ';')
	df = remove_ignored_classes(df)
	df = remove_duplicates(df)
	df = remove_rare_classes(df)
	df = remove_ignored_columns(df)
	df = df.fillna(0)
	return df

Na oczyszczenie danych składały się 4 fazy.
1 - Usunięcie ignorowanych klas ze zbioru danych:

In [ ]:
def remove_ignored_classes(df):
	ignored_classes = {'DA','DC','DT', 'DU',
		'DG', 'DI','UNK', 'UNX',
		'UNL', 'PR', 'PD', 'Y1',
		'EU', 'N', '15P', 'UQ',
		'PX4'}
	return df[~df.res_name.isin(ignored_classes) & ~df.res_name.isnull()]

2 - Usunięcie obserwacji, które są zwielokrotnione:

In [ ]:
def remove_duplicates(df):
	return df.drop_duplicates(subset = ['res_name', 'pdb_code'])

3 - Usunięcie obserwacji, które należą do klas występujących zbyt rzadko (mniej niż 5 wystąpień):

In [ ]:
def remove_rare_classes(df):
	class_sizes = df.groupby(['res_name']).size()
	rare_classes = set(class_sizes[class_sizes < 5].index)
	return df[~df.res_name.isin(rare_classes)]

4 - Usunięcie kolumn niedostępnych w klasyfikowanym zbiorze testowym:

In [ ]:
def remove_ignored_columns(df):
	ignored_columns = ['title', 'pdb_code', 'res_id', 'chain_id',
		'local_BAa', 'local_NPa', 'local_Ra', 'local_RGa',
		'local_SRGa', 'local_CCSa', 'local_CCPa', 'local_ZOa',
		'local_ZDa', 'local_ZD_minus_a', 'local_ZD_plus_a', 'local_res_atom_count',
		'local_res_atom_non_h_count', 'local_res_atom_non_h_occupancy_sum',
		'local_res_atom_non_h_electron_sum', 'local_res_atom_non_h_electron_occupancy_sum',
		'local_res_atom_C_count', 'local_res_atom_N_count', 'local_res_atom_O_count',
		'local_res_atom_S_count', 'dict_atom_non_h_count', 'dict_atom_non_h_electron_sum',
		'dict_atom_C_count', 'dict_atom_N_count', 'dict_atom_O_count', 'dict_atom_S_count',
		'fo_col', 'fc_col', 'weight_col', 'grid_space',
		'solvent_radius', 'solvent_opening_radius', 'part_step_FoFc_std_min',
		'part_step_FoFc_std_max','part_step_FoFc_std_step', 'resolution_max_limit']
	return df.drop(ignored_columns, axis = 1)

### Wyszukanie najlepszego klasyfikatora
Do klasyfikacji wykorzystano algorytm RandomForest. W tym kroku należało znaleźć najlepsze parametry tego klasyfikatora za pomocą metody grid search. Znaleziony klasyfikator był oceniany miarą weighted recall.

In [ ]:
score_measure = 'recall_weighted'
tune_parameters = {
	'n_estimators': [130, 140, 150, 160, 170, 180],
	'max_features': ['sqrt', 'log2'] }

def search(df, classes):
	df = df.drop('res_name', axis = 1)
	X_train, X_test, y_train, y_test = create_train_test(df, classes)
	rfc = create_random_forest_classifier()
	clf = GridSearchCV(
		estimator = rfc,
		param_grid = tune_parameters,
		scoring = score_measure)
	clf.fit(X_train, y_train)
	best_rfc = clf.best_estimator_
	print 'Best params: ', clf.best_params_
	y_true, y_pred = y_test, best_rfc.predict(X_test)
	print 'Score (' + score_measure + '): ', recall_score(y_true, y_pred, average='weighted')
	return best_rfc

Wyniki procedury dla dwóch klasyfikatorów pokazano w tabeli:

| klasyfikator | znalezione parametry | weighted recall |
|--------------|----------------------|-----------------|
| klasy oryginalne | `{'max_features': 'sqrt', 'n_estimators': 160}` | 0.420 |
| klasy grupowe | `{'max_features': 'sqrt', 'n_estimators': 160}` | 0.470 |

Znalezione klasyfikatory zostały zserializowane do plików za pomocą biblioteki `joblib` z pakietu `sklearn.externals`. Spakowane pliki z klasyfikatorami znajdują się w archiwach `classifiers/es.7z` (klasyfikator operujący na oryginalnych klasach) i `classifiers/esg.7z` (klasyfikator operujący na klasach grupowych).

### Klasyfikacja zbioru testowego
W ostatnim kroku poddawano klasyfikacji zbiór testowy dany w pliku `test_data.txt`.

In [ ]:
def classify(df, es):
	file = 'test_data.txt'
	tdf = read_data(file, ',')
	tdf = tdf[list(set(tdf.columns).intersection(df.columns))]
	tdf = tdf.fillna(0)
	return es.predict(tdf)

Wyniki klasyfikacji dla klas oryginalnych znajdują się w pliku `results/classify.txt`, a dla klas grupowych w pliku `results/classifyg.txt`.